# A First Taste

## States and Observations.

We calculate an expectations of any event $M$ that has take the values $\mu_1,\mu_2,..,\mu_k$ as


$$ \begin{equation}
\langle M \rangle = \sum_{i = 0}^k p_k\mu_k
\end{equation}
$$, where $p_k$ is the probabilites of each event.

In [42]:
import pennylane as qml
import numpy as np
#from pennylane import numpy as np
events_names = ['event_1','event_2','event_3','event_4'] #events names are values of events_names_M
events_numbers = [1,2,3,4]
events_probabilites = [0.2,0.2,0.2,0.4]
events_dict = dict(zip(events_names,events_numbers))
events_probabilites_dict = dict(zip(events_names,events_probabilites))
expectations_of_event_M = np.sum([events_probabilites[index] * events_numbers[index] for index in range(len(events_names))])
print(expectations_of_event_M)

#The result of 2.8 indicates that Event_3 and Event 4 are most likely to contains the information about the quantum state.


2.8000000000000003


So far we have four events and their probabilties. Upon calculating the expectation value, we found that particle on this system is most likely to be contained toward right hand then the left hand side of the system. Dr. Schuld claims that upon calcuating the expected value, we need two steps to arrive at the quantum system with K different possible configuration, measurements or outcomes. Remember here, we are at a single system and we have the chance to get $K$ outcomes. However, it is not clear yet, if this $K$ is same as the number of events $k$. 
1. Rewrite the probabilites and the outcomes as matrices and vector.
$$ \begin{equation}
q = \begin{pmatrix} \sqrt{p_1} \\ . \\ . \\ \sqrt{p_k} \end{pmatrix} = \sqrt{p_1} \begin{pmatrix} 1 \\ 0 \\ . \\ . \\0 \end{pmatrix} + ..... +\sqrt{p_k} \begin{pmatrix} 0 \\ 0 \\ . \\ . \\1 \end{pmatrix}
\end{equation}
$$
2. Replace real positive probabilities with complex amplitudes.

In [43]:
# Lets do step 1. 
# Writing prpbabilites as vectors.
q = np.asarray([np.sqrt(p) for p in events_probabilites])

matrix_length = len(events_names)
M = np.zeros((matrix_length,matrix_length),float)
np.fill_diagonal(M,events_numbers)
print(M)



[[1. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 4.]]


We can redefine the Expected value of an event as,
$$
\begin{equation}
\langle M \rangle = q^T M q = \sum_{i = 0}^k p_k\mu_k
\end{equation}
$$
where $M$ is a diagonal matrix with q or $\mu_i$ value along its diagonal.

In [44]:
expected_value = np.dot(np.dot(q.T,M),q) #This is a vectorized version of the above equation, and it is much faster than the above equation.
print(expected_value)

2.8


Lets define a basis vector $(1,0,....,0)^T$ that forms a subspace of $\mathbb{R}^k$ that is associated with first event $event_1$.

In [45]:
basis_vec_of_first_event = np.asarray([1 if i == 0 else 0 for i in range(len(events_names))])
outer_product_of_a  = np.outer(basis_vec_of_first_event,basis_vec_of_first_event)
print(np.dot(basis_vec_of_first_event,q))


0.4472135954999579


### Eigenvalue and Eigen Vector

In [46]:
eigen_val,eigen_vec = np.linalg.eig(M)
print(f"Eigenvalues of M: {eigen_val}")
print(f"Eigenvectors of M: {eigen_vec}")

#Lets check for the equation, M*v = w*v
v1_1 = np.asarray([1 if i == 0 else 0 for i in range(len(events_names))])
v2_2 = np.asarray([1 if i == 1 else 0 for i in range(len(events_names))])
v3_3 = np.asarray([1 if i == 2 else 0 for i in range(len(events_names))])
v4_4 = np.asarray([1 if i == 3 else 0 for i in range(len(events_names))])
v1,v2,v3,v4 = eigen_vec
q1,q2,q3,q4 = q.copy()

Eigenvalues of M: [1. 2. 3. 4.]
Eigenvectors of M: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


Above we define vectors $v1,..,v_k$ (for us k is 4), as an orthonormal eigenvectors of $M$. It fulfil $Mv_k  = \mu_k v_k$. \
We can define an amplitude vector $\alpha$ from $q$ such that 
$$
\langle M \rangle  = \alpha^{\dagger} M \alpha = \sum_{k = 1}^K | (v_k^{\dagger} \alpha )|^2 \mu_k
$$

In [47]:
l = np.dot(M,v1)
r = np.dot(events_numbers[0],v1)
a = np.asarray([np.sqrt(p) for p in events_probabilites],complex) #Complex amplitudes
M_via_v_and_mu = sum([np.dot(np.power(abs(np.dot(np.conj(eigen_vec[i]),a)),2),events_numbers[i]) for i in range(len(events_names))])
m_via_a = np.dot(np.dot(np.conj(a),M),a)
print(expectations_of_event_M,expected_value,M_via_v_and_mu,m_via_a)



2.8000000000000003 2.8 2.8 (2.8+0j)


Thus we have calculated an expected value using 4 methods.
1. $$ \begin{equation*} \langle M \rangle = \sum_{i = 0}^k p_k\mu_k,\text{ where $p_k$ is the probability of even $E_k$ and $\mu_k$ is an event value.}  \end{equation*} $$ 
2. $$ \begin{equation*} \langle M \rangle = q^T M q = \sum_{i = 0}^k p_k\mu_k \text{, where $q$ is a vector of $\sqrt{p_k}$ and $M$ is a diagonal matrix with $\mu_k$ on its diagonal.} \end{equation*} $$
3. $$\langle M \rangle  = \alpha^{\dagger} M \alpha \text{, where $\alpha$ is a complex vector of amplitudes. It is derived from $q$.}$$
4. $$\langle M \rangle  = \sum_{k = 1}^K | (v_k^{\dagger} \alpha )|^2 \mu_k \text{, where $v_i$ is an orthonormal eigen vectors of $M$.}$$


In [48]:
x = np.random.randn(2,1) #Get 10 random numbers from a normal distribution.
print(x)
k = np.sqrt(np.sum(x**2)) #Calculate the norm of the vector.
x_new = x/k #Normalize the vector.
sum([i**2 for i in x_new]) #Check that the norm is 1.


[[-1.53384357]
 [ 0.60144961]]


array([1.])

# 3.2 

## 3.2.2 Bits and Qubits


In [49]:
ket_0 = np.asarray([[1],[0]])
ket_1 = np.asarray([[0],[1]])
alpha_0 = 0.8 + 0.j
alpha_1 = 0.6 + 0.j
bra_0 = np.conj(ket_0).T
bra_1 = np.conj(ket_1).T
alpha = np.asarray([[alpha_0],[alpha_1]])
alpha_conj = np.conj(alpha).T



Geometric representation of a qubit.
$$
\ket{\psi} = e^{i \gamma}\big(cos\frac{\theta}{2} \ket{0} + e^{i \phi} sin\frac{\theta}{2}\ket{1}\big)
$$

In [50]:
def calculate_psi(gamma,psi,theta):
    state_ = np.exp(1j * gamma) * (np.cos(theta/2) * ket_0 + np.exp(1j * phi) * np.sin(theta/2) * ket_1)
    return state_

## Quantum Gates

In [51]:
X_gate = np.asarray([[0,1],[1,0]])
Y_gate = np.asarray([[0,-1j],[1j,0]])
Z_gate = np.asarray([[1,0],[0,-1]])
S_gate = np.asarray([[1,0],[0,1j]])
T_gate = np.asarray([[1,0],[0,np.exp(1j * np.pi/4)]])
Hadamard = np.asarray([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
R = np.asarray([[1,0],[0,np.exp(-1j * np.pi/4)]])
CNOT = np.asarray([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
SWAP = np.asarray([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
Toffoli = np.asarray([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],
                        [0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],
                        [0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])
Toffoli

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0]])

### Entagling Circuit
## Bell states
Bell states are the quantum states of two qubits whose spin can not be separated as spin of individual qubits. There are four bell states.
$$
1. \ket{\Psi_+} = \frac{1}{\sqrt{2}}(\ket{00} + \ket{11}) \\
2. \ket{\Psi_-} = \frac{1}{\sqrt{2}}(\ket{00} - \ket{11}) \\
3. \ket{\Phi_+} = \frac{1}{\sqrt{2}}(\ket{01} + \ket{10}) \\
4. \ket{\Phi_-} = \frac{1}{\sqrt{2}}(\ket{01} - \ket{10}) \\
$$

In [52]:
bell_state_psi_plus = 1/np.sqrt(2) * (np.tensordot(ket_0,ket_0,axes=([1],[1])) + np.tensordot(ket_1,ket_1,axes=([1],[1])))
bell_state_psi_minus = 1/np.sqrt(2) * (np.tensordot(ket_0,ket_0,axes=([1],[1])) - np.tensordot(ket_1,ket_1,axes=([1],[1])))
bell_state_phi_plus = 1/np.sqrt(2) * (np.tensordot(ket_0,ket_1,axes=([1],[1])) + np.tensordot(ket_1,ket_0,axes=([1],[1])))
bell_state_phi_minus = 1/np.sqrt(2) * (np.tensordot(ket_0,ket_1,axes=([1],[1])) - np.tensordot(ket_1,ket_0,axes=([1],[1])))
print("bell_state_psi_plus:\n",bell_state_psi_plus)
print()
print("bell_state_psi_minus:\n",bell_state_psi_minus)
print()
print("bell_state_phi_plus:\n",bell_state_phi_plus)
print()
print("bell_state_phi_minus:\n",bell_state_phi_minus)
print()


bell_state_psi_plus:
 [[0.70710678 0.        ]
 [0.         0.70710678]]

bell_state_psi_minus:
 [[ 0.70710678  0.        ]
 [ 0.         -0.70710678]]

bell_state_phi_plus:
 [[0.         0.70710678]
 [0.70710678 0.        ]]

bell_state_phi_minus:
 [[ 0.          0.70710678]
 [-0.70710678  0.        ]]



In [53]:
# Lets create Bell states via pennylane

dev = qml.device("default.qubit",wires = 2, shots = 100)
@qml.qnode(dev)
def bell_state_circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1)) #Return the expectation value of the PauliZ gate.
results = bell_state_circuit()
#Pennylane results will be in -1 and 1. Here, -1 means state 1 and 1 means state 0.
results = np.where(results == 1,0,results)
results = np.where(results == -1,1,results)
print (np.all(results[0] == results[1]))


True


### Pauli Rotations
For geometric interpretation, there are 3 single qubit gates handy. Below I list them all.
$$
1. \text{Rotation along x-axis}, \mathbb{R}_x(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}), -i\sin(\frac{\theta}{2}) \\ \\
-i\sin(\frac{\theta}{2}),cos(\frac{\theta}{2}) \end{pmatrix} \\ 

2. \text{Rotation along y-axis}, \mathbb{R}_y(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}), -\sin(\frac{\theta}{2}) \\ \\
\sin(\frac{\theta}{2}),cos(\frac{\theta}{2}) \end{pmatrix} \\ \\ \\
3. \text{Rotation along z-axis}, \mathbb{R}_z(\theta) = \begin{pmatrix} e^{(-i \pi\frac{\theta}{2})}, 0 \\ \\
0,e^{(i \pi\frac{\theta}{2})} \end{pmatrix}
$$

In [54]:
def pauli_rotations(angle,x = False, y = False, z = False):
    if x:
        return np.asarray([[np.cos(angle/2), -1 * 1j * np.sin(angle/2)],[-1 * 1j*np.sin(angle/2), np.cos(angle/2)]])
    elif y:
        return np.asarray([[np.cos(angle/2), -np.sin(angle/2)],[np.sin(angle/2), np.cos(angle/2)]])
    elif z:
        return np.asarray([[np.exp( 1j* (-1 * angle/2)),0],[0,np.exp((0 + 1j)*angle/2)]])


In [55]:
## Normalize the vector to unit length.
x   = np.asarray([0.1,-0.6,1.0])
norm_of_x = np.round(np.sqrt(np.sum(x**2)),3)
print(f"The norm is: {norm_of_x}")
x_new = x/norm_of_x
print(f"The normalized vector is: {x_new}")
print(sum(i**2 for i in x_new))



The norm is: 1.17
The normalized vector is: [ 0.08547009 -0.51282051  0.85470085]
1.000803564906129


### Time-Evolution Encoding

In encoding the input into quantum states, we often prefer to use  $U(x)  = e^{iH}$ where H is Hamiltonian. 

For most of the Quantum Machine Learning, $H$ is often a Pauli Rotation gates with $H = \frac{1}{2} \sigma_a, a \in \{x,y,z\}$,
We can apply $R_y(\theta)\ket{0}$ for a entry $(-0.53 )\rightarrow R_y(-0.53)\ket{0} = \cos \frac{-0.53}{2} \ket{0} + \sin \frac{-0.53}{2} \ket{1}$

We have a matrix,
$$
A = \begin{pmatrix} 0.085, -0.53 \\ 0.85, 0.00 \end{pmatrix}
$$
We can define a Hamiltonian as, 
$$
H = \begin{pmatrix} 0, A \\
A^{\dagger}, 0
\end{pmatrix}
$$

In [56]:
#Let's define this Hamiltonian
x_new_n = np.append(x_new,0.00)
x_new_n = x_new_n.reshape(2,2)
x_new_n


array([[ 0.08547009, -0.51282051],
       [ 0.85470085,  0.        ]])

In [67]:
H = np.asarray([[0,0,0.073,-0.438],[0,0,0.73,0],[0.073,0.73,0,0],[-0.438,0,0,0]])
print(H)
eigen_val,eigen_vec = np.linalg.eig(H)
eigen_val

[[ 0.     0.     0.073 -0.438]
 [ 0.     0.     0.73   0.   ]
 [ 0.073  0.73   0.     0.   ]
 [-0.438  0.     0.     0.   ]]


array([ 0.73563287,  0.43464616, -0.43464616, -0.73563287])

In [73]:
# Calculating the eigen value and eigen vector of the Hadamard gate.
eigen_val_H,eigen_vec_H = np.linalg.eig(Hadamard)
np.tensordot(Hadamard,eigen_val_H,axes=([1],[0]))

array([7.85046229e-17, 1.41421356e+00])

In [79]:
np.round(np.dot(np.conj(Hadamard),Hadamard),2)

array([[ 1., -0.],
       [-0.,  1.]])